In [153]:
import numpy as np
import pandas as pd 
import geopandas as gpd
import matplotlib.pyplot as plt
from census import Census

In [154]:
c = Census("your_key_here")

In [155]:
pd.set_option('display.max_columns',50)

In [156]:
pop_df = pd.read_excel('./PopulaitonProjections.xlsx')

In [179]:
#parcel_df = pd.read_excel("ParcelBlockGroup_Joined_7_18_2021.xlsx")
bg_df = pd.read_excel('./Data/7_11_2021_Exports/BlockGroupExport.xlsx')
#build_df = pd.read_excel('./Data/7_11_2021_Exports/BuildingFootprintPoints.xlsx')

In [158]:
parcel_df.dropna(subset=['GEOID'],inplace=True)
parcel_df['GEOID'] = parcel_df['GEOID'].astype(int)

In [159]:
parcel_df.columns

Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'FOLIO', 'TYPE', 'PIN', 'DOR_C',
       'OWNER', 'ADDR_1', 'ADDR_2', 'CITY', 'STATE', 'ZIP', 'COUNTRY', 'SUB',
       'SITE_ADDR', 'SITE_CITY', 'SITE_ZIP', 'LEGAL1', 'LEGAL2', 'LEGAL3',
       'LEGAL4', 'DBA', 'STRAP', 'tBEDS', 'tBATHS', 'tSTORIES', 'tUNITS',
       'tBLDGS', 'TAXDIST', 'JUST', 'LAND', 'BLDG', 'EXF', 'ACT', 'EFF',
       'HEAT_AR', 'ASD_VAL', 'TAX_VAL', 'MUNI', 'SD1', 'SD2', 'TIF', 'BASE',
       'S_DATE', 'VI', 'S_AMT', 'ACREAGE', 'NBHC', 'Edit_dt', 'SLR_tag',
       'SLR_1', 'SLR_2', 'SLR_3', 'SLR_Tag_8', 'SLR_BG', 'SLR_4', 'SLR5',
       'SLR_6', 'SLR_7', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON', 'SLRtag', 'SLR8', 'SLR4_BG', 'SLR2BG', 'SLR3BG', 'SLR1BG',
       'SLR5_BG', 'SLR6_BG', 'SLR7_BG', 'Shape_Length', 'Shape_Area'],
      dtype='object')

In [160]:
#parcel_df[parcel_df['SLR_2']==1]['JUST']

In [161]:
#parcel_df[parcel_df['SLR5']==1]

In [162]:
#DOR Codes identifying building type form property appraiser
dor_codes = pd.read_excel('./_DOR_Code_List.xlsx')
#Residential codes
res_codes=["0000","0006","0010","0020","0030",
           "0100", "0106","0300","0310", "0320", 
           "0330", "0340", "0340","0396", "0397",
           "0398", "0399", "0400","0400", "0400",
           "0500", "0700","0800",
          ]
#Mobile-home codes
mh_codes = ["0008",'0200',"0508",]
#Retirement homes
retirement_codes = ['0610','0611','0620','0621', 
                    "0630", "0631", "0640", "0641",
                    "0650", "0660", "0670", "0680"]
#Commercial property codes
com_codes = list(dor_codes.iloc[51:286,0])

In [163]:
#Get the median home vale by tract from ACS 5 year data
home_val = c.acs5dp.get(['NAME',"GEO_ID",'DP04_0080E','DP04_0089E'], geo={'for':"tract",'in':'state:12 county:057'}, year=2019)
home_val = pd.DataFrame(home_val)
home_val.columns = ['NAME', 'GEO_ID', 'OwnOc_HomeVal', 'MedHomeVal', 'state', 'county','tract']
home_val = home_val[['OwnOc_HomeVal', 'MedHomeVal','tract']]
home_val['tract'] = home_val['tract'].astype(float)

In [164]:
parcel_df = parcel_df.merge(home_val, how='left',left_on="TRACTCE",right_on='tract')

In [165]:
# Calculate the percentage of parcels impacted by SLR in each tract for the 2ft and 5ft scenarios
collect_list = []
for geo_id in parcel_df['TRACTCE'].unique():
    tempdf = parcel_df[parcel_df['TRACTCE']==geo_id]
    two_ft_pct = tempdf['OBJECTID'][tempdf['SLR_2']==1].count()/tempdf['OBJECTID'].count()
    five_ft_pct = tempdf['OBJECTID'][tempdf['SLR5']==1].count()/tempdf['OBJECTID'].count()
    dic = {'tract':geo_id,'2_ft_pct':two_ft_pct,'5_ft_pct':five_ft_pct}
    collect_list.append(dic)
tract_impact = pd.DataFrame(collect_list)

In [166]:
tract_impact = tract_impact.merge(home_val,how='left',on='tract')

In [167]:
proj = pd.read_csv("housing_unit_and_populaiton_projections.csv")

In [168]:
proj['tract'] = proj['GEO_ID'].astype(str).apply(lambda x: x[7:-1])
proj['tract'] = proj['tract'].astype(int)

In [169]:
proj = proj.merge(tract_impact,how='left', on='tract')

In [170]:
#BRV Calculaiton for 2070 HU projections
proj['2_ft_BRV'] = (proj['HU_2070']*proj['MedHomeVal'])*proj['2_ft_pct']
proj['5_ft_BRV'] = (proj['HU_2070']*proj['MedHomeVal'])*proj['5_ft_pct']

In [171]:
parcel_df.columns

Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'FOLIO', 'TYPE', 'PIN', 'DOR_C',
       'OWNER', 'ADDR_1', 'ADDR_2', 'CITY', 'STATE', 'ZIP', 'COUNTRY', 'SUB',
       'SITE_ADDR', 'SITE_CITY', 'SITE_ZIP', 'LEGAL1', 'LEGAL2', 'LEGAL3',
       'LEGAL4', 'DBA', 'STRAP', 'tBEDS', 'tBATHS', 'tSTORIES', 'tUNITS',
       'tBLDGS', 'TAXDIST', 'JUST', 'LAND', 'BLDG', 'EXF', 'ACT', 'EFF',
       'HEAT_AR', 'ASD_VAL', 'TAX_VAL', 'MUNI', 'SD1', 'SD2', 'TIF', 'BASE',
       'S_DATE', 'VI', 'S_AMT', 'ACREAGE', 'NBHC', 'Edit_dt', 'SLR_tag',
       'SLR_1', 'SLR_2', 'SLR_3', 'SLR_Tag_8', 'SLR_BG', 'SLR_4', 'SLR5',
       'SLR_6', 'SLR_7', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON', 'SLRtag', 'SLR8', 'SLR4_BG', 'SLR2BG', 'SLR3BG', 'SLR1BG',
       'SLR5_BG', 'SLR6_BG', 'SLR7_BG', 'Shape_Length', 'Shape_Area',
       'OwnOc_HomeVal', 'MedHomeVal', 'tract'],
      dtype='object')

In [172]:
res_parcels = parcel_df[~(parcel_df["DOR_C"].isin(res_codes)) | (parcel_df["DOR_C"].isin(retirement_codes))]

In [173]:
#Get the median "Heated Area" for each blockgroup and join to projection df
med_heat_area = res_parcels.groupby('GEOID').median()['HEAT_AR']
pd.DataFrame(med_heat_area).reset_index(inplace=True)
proj = proj.merge(med_heat_area, how='left',left_on="GEO_ID",right_on="GEOID")

In [177]:
#Calculate disruption cost based on $0.88 standard value per sqft 
proj['disp_cost_2ft'] = (proj['HEAT_AR'] * proj['HU_2070'] *proj['2_ft_pct'] * .88)
proj['disp_cost_5ft'] = (proj['HEAT_AR'] * proj['HU_2070'] *proj['5_ft_pct'] * .88) 

In [202]:
#BRV Calculaiton for 2070 HU projections
proj['2_ft_BRV_2020_HU'] = (proj['HU_2020']*proj['MedHomeVal'])*proj['2_ft_pct']
proj['5_ft_BRV_2020_HU'] = (proj['HU_2020']*proj['MedHomeVal'])*proj['5_ft_pct']
#Calculate disruption cost based on $0.88 standard value per sqft 
proj['disp_cost_2ft_2020_HU'] = (proj['HEAT_AR'] * proj['HU_2020'] *proj['2_ft_pct'] * .88)
proj['disp_cost_5ft_2020_HU'] = (proj['HEAT_AR'] * proj['HU_2020'] *proj['5_ft_pct'] * .88) 

In [194]:
#Scenario filter
slr2_bgs = list(bg_df[bg_df["SLR2BG"]==1]['GEOID'])
slr5_bgs = list(bg_df[bg_df["SLR5_BG"]==1]['GEOID'])
slr2_filt= [x for x in slr2_bgs]
slr5_filt= [x for x in slr5_bgs]

In [199]:
proj['2ftTag'] = np.where(proj['GEO_ID'].isin(slr2_filt), 1,0)

In [200]:
proj['5ftTag'] = np.where(proj['GEO_ID'].isin(slr5_filt), 1,0)

In [203]:
proj.to_excel('projection_and_costs.xlsx')